# Chatbot
1. LLM powered chatbot
2. Remebers the pervious interactions and conversations like other llms.

In [3]:
#using groq 

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
#model laoding
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [9]:

from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi, I am Darshan Jain and I am an AI Engineer")])

AIMessage(content="Hi Darshan Jain! It's nice to meet you.\n\nThat's great that you're an AI Engineer. It's a fascinating field with so much potential. \n\nWhat kind of AI work are you involved in?  Do you have any projects you're particularly excited about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 22, 'total_tokens': 87, 'completion_time': 0.118181818, 'prompt_time': 0.002138446, 'queue_time': 0.255497103, 'total_time': 0.120320264}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2b4cf532-1e5c-474c-ba1f-3f26ba8dd426-0', usage_metadata={'input_tokens': 22, 'output_tokens': 65, 'total_tokens': 87})

In [10]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Darshan Jain and I am an AI Engineer"),
        AIMessage(content="Hi Darshan Jain! It's nice to meet you.\n\nThat's great that you're an AI Engineer. It's a fascinating field with so much potential. \n\nWhat kind of AI work are you involved in?  Do you have any projects you're particularly excited about?\n"),
        HumanMessage(content="Hey , What is my name and what do i do?")
    ]
)

AIMessage(content="You told me your name is Darshan Jain and that you are an AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or ask me? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 108, 'total_tokens': 149, 'completion_time': 0.074545455, 'prompt_time': 0.005142099, 'queue_time': 0.24720503099999996, 'total_time': 0.079687554}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0c4fe23f-2bd3-4e78-8e09-9dedbbfad0b1-0', usage_metadata={'input_tokens': 108, 'output_tokens': 41, 'total_tokens': 149})

### Message history

#we can use a message history class to wrap our model and make it statefull.

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


#get_session_history will check wheter the session_id is present in the store (dictionary) or not . 
# if it is present in the store than all the previous chats history will be initialized for that session and it will return the session_id of the store.
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [15]:
config={"configurable":{"session_id":"chat1"}}

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Darshan Jain and I am an AI Engineer")],
    config=config

)

In [18]:
with_message_history.invoke(
    [HumanMessage(content="Hello, What is my name and what do i do?")],
    config=config,
)

AIMessage(content="You are Darshan Jain, and you are an AI Engineer!  \n\nIs there anything else you'd like to know or talk about? 😊  I'm here to help! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 246, 'total_tokens': 288, 'completion_time': 0.076363636, 'prompt_time': 0.021364257, 'queue_time': 0.261677712, 'total_time': 0.097727893}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--46825af0-a52f-41b3-8638-df0240ee8998-0', usage_metadata={'input_tokens': 246, 'output_tokens': 42, 'total_tokens': 288})

In [21]:
#change the config----->change the session_id

config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hello, What is my name and what do i do?")],
    config=config1,
)

response.content

"Since I'm a large language model, I don't have access to any information about you, including your name or what you do. \n\nIf you'd like to tell me, I'm happy to know!  😊  \n\n"

In [23]:
#Since i have changed the session_id ...the llm have no information about the past conversations.
# Because of with_message_history function.

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, my name is Jay modi")],
    config=config1,
)

response.content

"Hello Jay Modi, it's nice to meet you! \n\nIs there anything I can help you with today? 😊  \n\n"

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, what is my name?")],
    config=config1,
)

response.content

'You told me your name is Jay Modi! \n\nIs there anything else I can help you with? 😊  \n'

# Prompt Templates

In [28]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [29]:
chain=prompt|model

In [30]:
chain.invoke({"messages":[HumanMessage(content="Hello,my name is Darshan Jain")]})

AIMessage(content="Hello Darshan Jain, it's nice to meet you!  \n\nHow can I be helpful today? 😊  \n\nI'm ready for your questions!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 33, 'total_tokens': 71, 'completion_time': 0.069090909, 'prompt_time': 0.002363327, 'queue_time': 0.24940194200000002, 'total_time': 0.071454236}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--32c4fcd7-b610-434c-a33d-34e8383ba62f-0', usage_metadata={'input_tokens': 33, 'output_tokens': 38, 'total_tokens': 71})

In [32]:
#chat message histort in chain
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [37]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hii,my name is Darshan K Jain")],
    config=config
)
response

AIMessage(content="Hello Darshan K Jain! \n\nIt's great to meet you. How can I be of service today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 128, 'total_tokens': 157, 'completion_time': 0.052727273, 'prompt_time': 0.007477349, 'queue_time': 0.25376188, 'total_time': 0.060204622}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--dbe46969-c245-43da-988d-284180a9f17d-0', usage_metadata={'input_tokens': 128, 'output_tokens': 29, 'total_tokens': 157})

In [38]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, what is my name?")],
    config=config,
)

response.content

'Your name is Darshan K Jain.  \n\nIs there anything else I can help you with?\n'

In [39]:
#Adding more complexity.

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model

In [41]:
response=chain.invoke({"messages":[HumanMessage(content="Hi, my name is Darshan K Jain and I am an AI Engineer.")],"language":"Hindi"})

In [42]:
response.content

'नमस्ते Darshan K Jain! मेरा नाम है Bard, और मैं एक आर्टिफिशियल इंटेलिजेंस असिस्टेंट हूँ।\n\nआपसे क्या पूछना चाहते हैं? मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूँगा।  \n\n'

In [43]:
response1=chain.invoke({"messages":[HumanMessage(content="Hi, my name is Darshan K Jain and I am an AI Engineer.")],"language":"Marathi"})

In [44]:
response1.content

'नमस्कार, दर्शन के जैन! मला आनंद वाटत आहे की तुम्ही मला भेटाल आहात. मी तुमच्या मदत करण्यासाठी तयार आहे. तुम्हाला काय मदत हवी आहे? \n\n'

Lets now wrap tgis more complicated chain in a Message history class. This time , because there are multiple keys in the input , we need to specify the correct key to use to save the chat history

In [45]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [47]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi,I am Darshan Jain")],"language":"Hindi"},
    config=config,
)
response.content

'नमस्ते Darshan Jain! \n\nमुझे आपकी मदद करने में खुशी हो रही है। आप क्या जानना चाहते हैं? 😊  \n'

In [48]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name ?")],"language":"Hindi"},
    config=config,
)
response.content

'आपका नाम Darshan Jain है। 😊  \n'

# Managing the conversation History

'trim_messages' is a helper to reduce how many messages we are sending to the model.
The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and wheter to allow partial message.

In [49]:
from langchain_core.messages import trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [50]:
messages= [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

c:\Users\PINKY\OneDrive\Desktop\Chatbots_Langcahin\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\PINKY\OneDrive\Desktop\Chatbots_Langcahin\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PINKY\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In 

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

result=chain.invoke(
{  
    "messages":messages + [HumanMessage(content="What ice cream do i like?")],
    "language":"English"
},
config=config,
)



In [65]:
result.content

'You like vanilla ice cream!  🍦\n'

In [ ]:
result=chain.invoke(
{  
    "messages":messages + [HumanMessage(content="What math problem did i asked?")],
    "language":"English"
},
config=config,
)
result.content

'You asked "what\'s 2 + 2".  😊\n\n\n\n\n'

In [68]:
#lets wrap this message history

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
result=chain.invoke(
{  
    "messages":messages + [HumanMessage(content="What is my name and what ice cream do i like?")],
    "language":"English"
},
config=config,
)
result.content

'You said your name is Bob, and you like vanilla ice cream!  🍦  \n'

In [70]:
result=chain.invoke(
{  
    "messages":messages + [HumanMessage(content="What math problem did i asked?")],
    "language":"English"
},
config=config,
)
result.content

'You asked "What is 2 + 2?". 😊 \n\n\n\nLet me know if you have any other questions!\n'